# File Uploads and Media

This notebook covers uploading files and working with media in Notion:
- Understanding the enhanced file upload API
- Uploading small files with progress tracking
- Importing files from external URLs
- Adding uploaded files to pages (images, PDFs, files)
- Using external media without uploading
- Error handling and best practices

## Prerequisites

Make sure you have set these environment variables:
- `NOTION_API_TOKEN` - Your Notion integration token
- `NOTION_TEST_PAGE_ID` - A test page ID where we can create test pages

## Setup: Load Dependencies and Initialize Client

In [ ]:
// Load the Kotlin Notion Client library from Maven Central
@file:DependsOn("it.saabel:kotlin-notion-client:0.2.0")

// Import necessary classes
import it.saabel.kotlinnotionclient.NotionClient
import it.saabel.kotlinnotionclient.models.files.*
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.delay

// Initialize the client
val apiToken = System.getenv("NOTION_API_TOKEN")
    ?: error("❌ NOTION_API_TOKEN environment variable not set")

val parentPageId = System.getenv("NOTION_TEST_PAGE_ID")
    ?: error("❌ NOTION_TEST_PAGE_ID environment variable not set")

val notion = NotionClient(apiToken)

println("✅ NotionClient initialized successfully!")

## Understanding File Uploads in Notion

The Kotlin Notion Client provides two file upload APIs:

1. **Enhanced API** (`notion.enhancedFileUploads`) - **Recommended**
   - Automatic single-part vs multi-part detection
   - Built-in progress tracking
   - Retry logic and error recovery
   - Simple, production-ready API

2. **Basic API** (`notion.fileUploads`) - **Advanced Control**
   - Manual control over upload parameters
   - Direct access to Notion's low-level API
   - For custom upload logic

In this notebook, we'll focus on the enhanced API which handles most use cases automatically.

### File Size Limits
- **Free workspaces**: 5 MiB per file
- **Paid workspaces**: 5 GiB per file
- **Multi-part threshold**: Files > 20 MiB automatically use multi-part upload

## Example 1: Simple File Upload with Progress Tracking

Let's start by uploading a small text file and tracking its progress.

In [2]:
// Create test content
val testContent = """
    Notion File Upload Example
    ==========================
    
    This file was uploaded using the Enhanced File Upload API.
    
    Features demonstrated:
    - Simple upload from byte array
    - Progress tracking with callbacks
    - Automatic content-type detection
    
    Timestamp: ${System.currentTimeMillis()}
""".trimIndent()

// Track progress updates
val progressUpdates = mutableListOf<String>()

// Configure upload with progress tracking
val options = FileUploadOptions(
    progressCallback = { progress ->
        val message = "${progress.status}: ${String.format("%.1f", progress.progressPercent)}%"
        progressUpdates.add(message)
        println("📊 $message")
    }
)

// Upload the file
val result = runBlocking {
    notion.enhancedFileUploads.uploadFile(
        filename = "example-document.txt",
        data = testContent.toByteArray(),
        options = options
    )
}

// Check the result
when (result) {
    is FileUploadResult.Success -> {
        println("\n✅ File uploaded successfully!")
        println("   File ID: ${result.uploadId}")
        println("   Filename: ${result.filename}")
        println("   Upload time: ${result.uploadTimeMs}ms")
        println("   Progress updates: ${progressUpdates.size}")
    }
    is FileUploadResult.Failure -> {
        println("\n❌ Upload failed: ${result.error.message}")
    }
}

📊 STARTING: 0.0%
📊 UPLOADING: 0.0%
📊 COMPLETED: 100.0%

✅ File uploaded successfully!
   File ID: 295c63fd-82ed-819b-9cb7-00b2124b5939
   Filename: example-document.txt
   Upload time: 2934ms
   Progress updates: 3


## Example 2: Using Uploaded Files in Pages

Once a file is uploaded, we can reference it in pages using file blocks. Let's create a page that includes the file we just uploaded.

In [ ]:
// Only proceed if the upload was successful
var uploadedFilePage: it.saabel.kotlinnotionclient.models.pages.Page? = null

if (result is FileUploadResult.Success) {
    val page = runBlocking {
        notion.pages.create {
            parent.page(parentPageId)
            title("File Upload Example")
            
            content {
                heading1("Uploaded File Demo")
                paragraph("This page demonstrates using an uploaded file in Notion.")
                
                divider()
                
                heading2("File Details")
                bullet("Filename: ${result.filename}")
                bullet("Upload time: ${result.uploadTimeMs}ms")
                bullet("File ID: ${result.uploadId}")
                
                divider()
                
                heading2("The Uploaded File")
                paragraph("Here's the file we uploaded:")
                
                // Reference the uploaded file
                fileFromUpload(
                    fileUploadId = result.fileUpload.id,
                    name = result.filename,
                    caption = "Uploaded using the Enhanced File Upload API"
                )
            }
            
            icon.emoji("📤")
        }
    }
    
    println("\n✅ Page created with uploaded file!")
    println("   Page ID: ${page.id}")
    println("   URL: ${page.url}")
    
    uploadedFilePage = page
} else {
    println("⚠️  Skipping page creation since upload failed")
}

## Example 3: Import File from External URL

You can also import files from external URLs without manually downloading them first. This is useful for importing publicly accessible files.

In [ ]:
// Import a PDF from an external URL
val externalResult = runBlocking {
    notion.enhancedFileUploads.importExternalFile(
        filename = "sample-pdf",
        externalUrl = "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf",
        contentType = "application/pdf"
    )
}

when (externalResult) {
    is FileUploadResult.Success -> {
        println("✅ External file imported successfully!")
        println("   File ID: ${externalResult.uploadId}")
        println("   Filename: ${externalResult.filename}")
        
        // Wait for the file to be fully processed
        runBlocking {
            delay(2000) // Give Notion time to process the external file
        }
        
        println("   File is ready to use")
    }
    is FileUploadResult.Failure -> {
        println("❌ Import failed: ${externalResult.error.message}")
    }
}

## Example 4: Create a Page with the Imported PDF

Let's create a page that displays the imported PDF.

In [ ]:
var importedPdfPage: it.saabel.kotlinnotionclient.models.pages.Page? = null

if (externalResult is FileUploadResult.Success) {
    val pdfPage = runBlocking {
        notion.pages.create {
            parent.page(parentPageId)
            title("External PDF Import Example")
            
            content {
                heading1("External File Import")
                paragraph("This page demonstrates importing a file from an external URL.")
                
                callout("🌐") {
                    text("External imports require HTTPS URLs and the file must be publicly accessible.")
                }
                
                divider()
                
                heading2("Imported PDF Document")
                paragraph("Here's the PDF imported from an external URL:")
                
                // Display the imported PDF
                pdfFromUpload(
                    fileUploadId = externalResult.fileUpload.id,
                    caption = "Sample PDF imported from W3C test files"
                )
            }
            
            icon.emoji("📄")
        }
    }
    
    println("\n✅ Page created with imported PDF!")
    println("   Page ID: ${pdfPage.id}")
    println("   URL: ${pdfPage.url}")
    
    importedPdfPage = pdfPage
} else {
    println("⚠️  Skipping PDF page creation since import failed")
}

## Example 5: Using External Media (Without Upload)

You can also embed images and videos from external URLs directly without uploading them to Notion. This is useful for content hosted elsewhere.

In [ ]:
val externalMediaPage = runBlocking {
    notion.pages.create {
        parent.page(parentPageId)
        title("External Media Examples")
        
        content {
            heading1("External Media Demo")
            paragraph("This page demonstrates embedding external media without uploading.")
            
            divider()
            
            heading2("External Image")
            paragraph("Images can be embedded directly from external URLs:")
            
            // Embed an external image
            image(
                url = "https://images.unsplash.com/photo-1551033406-611cf9a28f67",
                caption = "Photo from Unsplash - embedded via external URL"
            )
            
            divider()
            
            heading2("External Video")
            paragraph("Videos can also be embedded from external sources:")
            
            // Embed an external video
            video(
                url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
                caption = "YouTube video embedded via external URL"
            )
            
            divider()
            
            callout("💡") {
                text("External media is not uploaded to Notion - it's embedded from the original URL. ")
                text("Make sure the URLs remain accessible!")
            }
        }
        
        icon.emoji("🎬")
    }
}

println("✅ Page created with external media!")
println("   Page ID: ${externalMediaPage.id}")
println("   URL: ${externalMediaPage.url}")

## Example 6: Error Handling

File uploads can fail for various reasons. Let's demonstrate how to handle errors gracefully.

In [ ]:
// Try to import from an invalid URL (non-HTTPS)
val errorResult = runBlocking {
    notion.enhancedFileUploads.importExternalFile(
        filename = "invalid-url-test",
        externalUrl = "http://example.com/file.pdf", // HTTP not allowed - must be HTTPS
        contentType = "application/pdf"
    )
}

// Handle the result
when (errorResult) {
    is FileUploadResult.Success -> {
        println("✅ Unexpected success: ${errorResult.uploadId}")
    }
    is FileUploadResult.Failure -> {
        println("❌ Expected failure occurred:")
        println("   Filename: ${errorResult.filename}")
        
        // Handle specific error types
        when (val error = errorResult.error) {
            is FileUploadError.ValidationError -> {
                println("   Error type: Validation Error")
                println("   Message: ${error.message}")
                println("   ℹ️  This is expected - HTTP URLs are not allowed, only HTTPS")
            }
            is FileUploadError.NetworkError -> {
                println("   Error type: Network Error")
                println("   Message: ${error.message}")
            }
            is FileUploadError.RateLimitError -> {
                println("   Error type: Rate Limit")
                println("   Retry after: ${error.retryAfterSeconds} seconds")
            }
            else -> {
                println("   Error type: ${error::class.simpleName}")
                println("   Message: ${error.message}")
            }
        }
    }
}

## Cleanup

Let's clean up by archiving the test pages we created.

In [ ]:
runBlocking {
    // Archive all test pages
    uploadedFilePage?.let { page ->
        try {
            notion.pages.archive(page.id)
            println("✅ Archived: File upload example page")
        } catch (e: Exception) {
            println("⚠️  Could not archive uploaded file page: ${e.message}")
        }
    }
    
    importedPdfPage?.let { page ->
        try {
            notion.pages.archive(page.id)
            println("✅ Archived: External PDF import page")
        } catch (e: Exception) {
            println("⚠️  Could not archive PDF page: ${e.message}")
        }
    }
    
    try {
        notion.pages.archive(externalMediaPage.id)
        println("✅ Archived: External media page")
    } catch (e: Exception) {
        println("⚠️  Could not archive media page: ${e.message}")
    }
}

println("\n🧹 Cleanup complete!")

## Summary

In this notebook, we explored file uploads and media in Notion:

### Key Operations
- **Upload files**: Use `notion.enhancedFileUploads.uploadFile()` for simple uploads
- **Import external files**: Use `importExternalFile()` for files hosted elsewhere
- **Embed external media**: Use `image(url, caption)` and `video(url, caption)` for external content
- **Track progress**: Use `FileUploadOptions` with progress callbacks
- **Handle errors**: Check `FileUploadResult` and handle specific error types

### Upload Methods Demonstrated
1. **From byte array**: Upload content directly from memory
2. **From external URL**: Import files from HTTPS URLs
3. **External embedding**: Embed media without uploading

### File Block Types
- **File blocks**: `fileFromUpload(fileUploadId, name, caption)` - Generic file display
- **PDF blocks**: `pdfFromUpload(fileUploadId, caption)` - Inline PDF viewer
- **Image blocks**: `imageFromUpload(fileUploadId, name, caption)` or `image(url, caption)` - Display images
- **Video blocks**: `videoFromUpload(fileUploadId, name, caption)` or `video(url, caption)` - Embed videos

### Best Practices
1. **Use enhanced API**: Handles most cases automatically with progress tracking
2. **Handle errors**: Always check `FileUploadResult` and handle failures
3. **External URLs**: Must use HTTPS (not HTTP)
4. **File size limits**: Free workspaces: 5 MiB, Paid: 5 GiB
5. **Progress tracking**: Use callbacks for better user experience
6. **Wait after import**: External files may need processing time

### File Size Limits
- **Free workspaces**: 5 MiB per file
- **Paid workspaces**: 5 GiB per file  
- **Multi-part threshold**: Files > 20 MiB use multi-part upload automatically
- **Multi-part minimum**: Each part must be >= 5 MiB (except last part)

### Error Types
- `ValidationError`: Invalid parameters or unsupported file type
- `FileSizeError`: File exceeds workspace limits
- `NetworkError`: Network connectivity issues
- `RateLimitError`: API rate limit exceeded
- `TimeoutError`: Operation took too long

## Next Steps

Continue exploring:
- **[docs/file-uploads.md](../docs/file-uploads.md)** - Comprehensive file upload documentation
- **[04-working-with-blocks.ipynb](./04-working-with-blocks.ipynb)** - More block types and operations
- **[06-advanced-queries.ipynb](./06-advanced-queries.ipynb)** - Complex filtering and pagination